In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import os

from tqdm.autonotebook import tqdm as tqdm

import neurokit2 as nk
from neurokit2.eda import eda_process


In [2]:
sys.path.append("'D:\CODING\Emotion_Recognition_Project\EDA-activities")
os.getcwd()

os.chdir("..")

from eda_helper import Processing

os.chdir("./notebooks")

---

In [3]:
def classifier_4_class(val, aro):
    """
    Classifies the input into one of four categories based on valence and arousal.
    :param val: Valence value
    :param aro: Arousal value
    :return: A string representing the category ('HH', 'HL', 'LH', 'LL')
    """
    if val > 5 and aro > 6.5:
        return "HH"  # High Valence, High Arousal
    elif val > 5 and aro <= 6.5:
        return "HL"  # High Valence, Low Arousal
    elif val <= 5 and aro > 6.5:
        return "LH"  # Low Valence, High Arousal
    else:  # val <= 5 and aro <= 6.5
        return "LL"  # Low Valence, Low Arousal


def classifier_2_class_arousal(aro):
    """
    Classifies the input into one of two categories based on arousal.
    :param aro: Arousal value
    :return: A string representing the category ('H', 'L')
    """
    if aro > 6.5:
        return "H"  #  High Arousal
    else:  # aro <= 6.5
        return "L"  # Low Arousal

Flow: *Load the Data* ==> *Apply the feature extraction* ==> *prepare the Data frame of labels* ==> *concat the features dataframe* ==> *Normalize the Data* ==> *write the data*

In [ ]:
for i in tqdm(range(1, 31), desc="Generating Neurokit Processed Data:"):
    sub_data = pd.read_csv(
        f"../CASE_full/CASE_full/data/interpolated/physiological/sub_{i}.csv"
    )[::50]
    sub_labels = pd.read_csv(
        f"../CASE_full/CASE_full/data/interpolated/annotations/sub_{i}.csv"
    )
    sub_analysis = eda_process(
        sub_data["gsr"], sampling_rate=1000 / 50, method="neurokit"
    )[0]

    # Create the Data Frame for video and labels from original subject dataset
    final_sub_data = pd.merge(
        pd.DataFrame(
            {
                "time": sub_data["daqtime"],
                "gsr": sub_data["gsr"],
                "video": sub_data["video"],
            }
        ),
        pd.DataFrame(
            {
                "valence": sub_labels["valence"],
                "arousal": sub_labels["arousal"],
                "time": sub_labels["jstime"],
            }
        ),
        on="time",
        how="inner",
    )

    # Merge the Neruokit analyzed Features
    final_sub_data = pd.concat(
        [final_sub_data, sub_analysis[["EDA_Clean", "EDA_Phasic", "EDA_Tonic"]]], axis=1
    )

    # Add the 4 class and 2 class arousal labels
    final_sub_data["class_4"] = final_sub_data.apply(
        lambda row: classifier_4_class(val=row["valence"], aro=row["arousal"]), axis=1
    )
    final_sub_data["class_2_arousal"] = final_sub_data.apply(
        lambda row: classifier_2_class_arousal(aro=row["arousal"]), axis=1
    )

    # Write the Data
    final_sub_data.to_csv(f"../preprocessed_data/sub_{i}.csv", index=False)
    print(len(final_sub_data))

Generating Neurokit Processed Data::   0%|          | 0/30 [00:00<?, ?it/s]

49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032
49032


---

## Working on Windowed Data From Here

Generating the windowed data for the Data of the video id: [5,6,7,8].

In [3]:
# Selecting the videos
data = pd.read_csv("../preprocessed_data/sub_1.csv")
data = data[data["video"].isin([5,6,7,8])]
data

,time,gsr,video,valence,arousal,EDA_Clean,EDA_Phasic,EDA_Tonic,class_4,class_2_arousal
15316,765800,4.948,5,5.0,5.001,4.949262,-0.007233,4.956495,LL,L
15317,765850,4.948,5,5.0,5.000,4.950045,-0.006510,4.956555,LL,L
15318,765900,4.956,5,5.0,5.000,4.949724,-0.006894,4.956618,LL,L
15319,765950,4.941,5,5.0,5.000,4.949175,-0.007509,4.956684,LL,L
15320,766000,4.948,5,5.0,5.000,4.949311,-0.007441,4.956752,LL,L
...,...,...,...,...,...,...,...,...,...,...
46627,2331350,6.959,8,0.5,9.500,6.945963,0.031642,6.914321,LH,H
46628,2331400,6.943,8,0.5,9.500,6.947660,0.031422,6.916238,LH,H
46629,2331450,6.943,8,0.5,9.500,6.949633,0.031482,6.918151,LH,H
46630,2331500,6.963,8,0.5,9.500,6.952043,0.031983,6.920061,LH,H


In [4]:
from eda_helper import Processing

In [7]:
def prepare_windows_with_labels(
    data,
    x_column="EDA_Phasic",
    y_columns=["arousal", "valence"],
    y_aggregator = "mean",
    window_size=100,
    window_gap=1,
):
    # prepare the x column
    x_df = Processing.generate_windows_df(data, window_size, window_gap, x_column)
    
    # prepare the y columns
    y_dfs = []
    for y_column in tqdm(y_columns, desc="Preparing Y Columns:"):
        y_df = Processing.generate_windows_df(data, window_size, window_gap, y_column)
        # Handle the aggregation based on the type of y_aggregator
        if isinstance(y_aggregator, str): # for standard functions
            y_df = y_df.agg(y_aggregator, axis=1)
        else:
            # For custom functions, use apply to avoid the FutureWarning
            y_df = y_df.apply(y_aggregator, axis=1)
        assert type(y_df) == pd.Series
        y_dfs.append(y_df)
    y_df = pd.concat(y_dfs, axis=1)
    y_df.columns = y_columns
    
    assert x_df.shape[0] == y_df.shape[0]
    # merge the x and y columns
    final_df = pd.concat([x_df,y_df], axis=1)
    
    
    return final_df

In [19]:
test = prepare_windows_with_labels(data,y_aggregator=lambda x: x.mean(), window_size=1000, window_gap=10)
test

generating window:   0%|          | 0/1166 [00:00<?, ?it/s]

preparing dataframe:   0%|          | 0/1166 [00:00<?, ?it/s]

Preparing Y Columns::   0%|          | 0/2 [00:00<?, ?it/s]

generating window:   0%|          | 0/1166 [00:00<?, ?it/s]

preparing dataframe:   0%|          | 0/1166 [00:00<?, ?it/s]

generating window:   0%|          | 0/1166 [00:00<?, ?it/s]

preparing dataframe:   0%|          | 0/1166 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,992,993,994,995,996,997,998,999,arousal,valence
0,-0.007233,-0.006510,-0.006894,-0.007509,-0.007441,-0.006344,-0.004470,-0.002445,-0.001062,-0.000864,...,0.010275,0.008484,0.004010,-0.001352,-0.005302,-0.006350,-0.004834,-0.002673,5.141530,6.412159
1,-0.001541,-0.001984,-0.001299,0.000297,0.001781,0.002320,0.001702,-0.000122,-0.003381,-0.007966,...,-0.003525,-0.004101,-0.004728,-0.006666,-0.009757,-0.011684,-0.010140,-0.005744,5.148399,6.432559
2,-0.012788,-0.016167,-0.016986,-0.015372,-0.012361,-0.009118,-0.006437,-0.004718,-0.004080,-0.004366,...,-0.005526,-0.008589,-0.007832,-0.002902,0.003421,0.007661,0.008121,0.005642,5.155256,6.452959
3,-0.005165,-0.006050,-0.006993,-0.008515,-0.011206,-0.014866,-0.018067,-0.018772,-0.015728,-0.009645,...,0.003343,0.006879,0.009599,0.009001,0.004394,-0.002442,-0.008262,-0.010377,5.161762,6.473359
4,-0.003178,0.000469,-0.000587,-0.005911,-0.013149,-0.019224,-0.021576,-0.019236,-0.013406,-0.006905,...,0.000977,0.003420,0.003423,0.002077,0.001111,0.001855,0.004171,0.006164,5.167885,6.493759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,0.007006,0.008394,0.007688,0.005164,0.002299,0.000631,0.000588,0.001251,0.001285,0.000269,...,-0.008030,-0.003020,0.000396,0.002740,0.004739,0.006983,0.009881,0.013422,9.125338,0.667743
1162,-0.000692,0.000313,0.004355,0.010318,0.015089,0.015391,0.010087,0.001209,-0.007132,-0.011168,...,0.020051,0.019413,0.018707,0.019049,0.020701,0.023237,0.025982,0.028161,9.144088,0.657443
1163,-0.009547,-0.003830,0.002577,0.006179,0.005049,-0.000092,-0.005913,-0.008196,-0.004688,0.003010,...,0.027219,0.026918,0.028548,0.031852,0.035676,0.038993,0.041527,0.043554,9.162838,0.647143
1164,0.010338,0.012887,0.009097,0.000747,-0.008393,-0.014402,-0.014828,-0.009704,-0.001520,0.005965,...,0.046588,0.044947,0.041489,0.036982,0.032918,0.030758,0.031047,0.033082,9.181588,0.636843


In [20]:
test.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,992,993,994,995,996,997,998,999,arousal,valence
888,-0.000570,-0.000291,-0.000494,-0.000736,-0.000172,0.001054,0.001577,0.000096,-0.003128,-0.006145,...,0.020719,0.021362,0.020380,0.017897,0.014232,0.010166,0.006979,0.005867,5.089513,5.045914
971,0.003698,-0.000541,-0.002270,-0.000529,0.002849,0.004512,0.002204,-0.003522,-0.009744,-0.013149,...,0.012140,0.012625,0.009336,0.003191,-0.003438,-0.007708,-0.007842,-0.004265,5.689108,4.238079
85,-0.004251,-0.005391,-0.004171,-0.001660,0.000406,0.000672,-0.001397,-0.005235,-0.009052,-0.010691,...,0.006259,0.005597,0.004604,0.003431,0.002638,0.002491,0.002281,0.000993,5.758000,5.379658
256,-0.001557,-0.004096,-0.005082,-0.003697,-0.000726,0.001855,0.002231,0.000336,-0.001748,-0.001367,...,0.007485,0.008622,0.008788,0.008199,0.006827,0.005032,0.004002,0.004946,6.003528,4.569692
383,0.031191,0.033272,0.034741,0.035142,0.034723,0.034544,0.035590,0.037612,0.039000,0.037917,...,0.060317,0.056429,0.052155,0.046773,0.040956,0.036233,0.033752,0.033471,9.125821,1.308326
